In [1]:
import os, sys
import numpy as np
import torch
import h5py
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [29]:
def chunks(folder_count, file, signal, chunk_size, slide_size=15, norm=None):
        data = []
        label = []
        f_label = []
        n_chunks = int((len(signal)-chunk_size)/slide_size) +  1
        for i in range(n_chunks):
            chunk = signal[int(i*slide_size) : int(chunk_size+(i*slide_size))]
            if norm =='inside_chunking':
                chunk = normalise_data(chunk)
            data.append(chunk)
            label.append([int(file)])
            f_label.append(folder_count)
        return data, label, f_label, n_chunks
    
def normalise_data(data, norm_type='min_max'):
    if norm_type=='z':
        m = np.mean(data) 
        sd = np.sqrt((sum(data-m)**2)/len(data))
        data = (data-m)/sd   
    elif norm_type=='min_max':
        data = (data - min(data))/(max(data)-min(data))
    return data

def load_data(fold, chunk_s, slide_s, norm=False, split_percent=0.9, dump=False, load_dump=False):
        if load_dump:
            with open('processed/{}_{}_ch{}_s{}.npy'.format(norm, fold, chunk_s, slide_s), 'rb') as f:
                return np.load(f, allow_pickle=True)
        data_train, label_train = [], []
        data_test, label_test = [], []
        train_f, test_f = [], []
        chunks_count, full_label = [], []
        train_set, test_set = [], []
        data_val, label_val = [], []
        folder_count = 0
        files = sorted(os.listdir('data'))
        f_check = {0:[], 1:[], 'l0':[], 'l1':[]}
        
        for i in range(len(files)):
            file_label_ = int(files[i][-5])
            with h5py.File('data/'+files[i], 'r') as f:
                    data = np.array(f['reconstructed_calcium'])
            if file_label_ == 1:
                f_check[1].append(files[i])
                f_check['l1'].append(len(data))
            elif file_label_ == 0:
                f_check[0].append(files[i])
                f_check['l0'].append(len(data))

        temp0 = list(zip(f_check['l0'], f_check[0]))
        temp0.sort(reverse=True)
        f_check['l0'], f_check[0] = zip(*temp0)  
        temp0 = list(zip(f_check['l1'], f_check[1]))
        temp0.sort(reverse=True)
        f_check['l1'], f_check[1] = zip(*temp0)
        f1, f2, f3, f4, f5 = [], [], [], [], []
        
        for i in range(len(f_check['l1'])):
            if i%5 == 0: f1.append(f_check[1][i])
            elif i%5 == 1: f2.append(f_check[1][i])
            elif i%5 == 2: f3.append(f_check[1][i])
            elif i%5 == 3: f4.append(f_check[1][i])
            elif i%5 == 4: f5.append(f_check[1][i])
        for i in range(len(f_check[0])):
            if i%5 == 0: f1.append(f_check[0][i])
            elif i%5 == 1: f2.append(f_check[0][i])
            elif i%5 == 2: f3.append(f_check[0][i])
            elif i%5 == 3: f4.append(f_check[0][i])
            elif i%5 == 4: f5.append(f_check[0][i])
                
        if fold == '1': train_set = f2 + f3 + f4 + f5; test_set = f1;
        if fold == '2': train_set = f1 + f3 + f4 + f5; test_set = f2;
        if fold == '3': train_set = f1 + f2 + f4 + f5; test_set = f3;
        if fold == '4': train_set = f1 + f2 + f3 + f5; test_set = f4;
        if fold == '5': train_set = f1 + f2 + f3 + f4; test_set = f5;

        for file in train_set:
            with h5py.File('data/'+file, 'r') as f:
                data = np.array(f['reconstructed_calcium'])
            if norm=='before_chunking': data = normalise_data(data);
            data_, label, f_label, c_c = chunks(folder_count, file=file[-5], signal=data, 
                                                chunk_size=chunk_s, slide_size=slide_s, norm=norm)
            data_train.extend(data_); label_train.extend(label); train_f.extend(f_label)
            folder_count += 1
            
        for file in test_set:
            with h5py.File('data/'+file, 'r') as f:
                data = np.array(f['reconstructed_calcium'])
            if norm=='before_chunking': data = normalise_data(data);
            data_, label, f_label, c_c = chunks(folder_count, file=file[-5], signal=data, 
                                                chunk_size=chunk_s, slide_size=slide_s, norm=norm)
            data_test.extend(data_); chunks_count.append(c_c); label_test.extend(label)
            test_f.extend(f_label); full_label.append(int(file[-5]))
            folder_count += 1

        data_val = data_train[int(len(data_train)*split_percent):]
        data_train = data_train[:int(len(data_train)*split_percent)]
        label_val = label_train[int(len(label_train)*split_percent):]
        label_train = label_train[:int(len(label_train)*split_percent)]
        train_f = train_f[:int(len(train_f)*split_percent)]
        if dump==True:
            with open('processed/{}_{}_ch{}_s{}.npy'.format(norm, fold, chunk_s, slide_s), 'wb') as f:
                np.save(f, np.array([data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label]))
        
        return data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label

In [23]:
for fold in ['1', '2', '3', '4', '5']:
    for ss in [15, 30, 45]:
        a = load_data(chunk_s=300, slide_s=ss, fold=fold, norm='inside_chunking', dump=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [26]:
a = 
data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label = a

load


In [4]:
class Net(nn.Module):
    def __init__(self, feat, k, s, p, z):
        super(Net, self).__init__()
        self.conv = nn.Conv1d(1, feat, kernel_size=k,stride=s)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(p)
        self.feat = feat
        self.z = z
        self.linear = nn.Linear(feat*z,2)
    def forward(self, x):
        c_in = x.permute(0, 2, 1)a, b
        out = self.conv(c_in)
        out = self.relu(out)
        c_out = self.pool(out)
        c_out = c_out.reshape(c_out.shape[0], self.feat*self.z)
        c_out = self.linear(c_out)
        return c_out

def train():   
    file_acc = {}
    correct, loss_ = 0, 0
    model.train()
    outs, labs = [], []
    for i, data in enumerate(trainloader, 0):
        inputs, labels, f = data
        optimizer.zero_grad()
        if cuda_enabled==True:
            inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        else:
            inputs, labels = inputs, labels.squeeze()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        comp = equals.squeeze().cpu().detach().numpy()
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        for j in range(len(f)):
            file = f[j].item()
            if file not in file_acc:
                file_acc[file] = [0, 0]
            if comp[j] == True:
                file_acc[file][0]+=1
                file_acc[file][1]+=1
            else:
                file_acc[file][1]+=1
    for i in file_acc.keys():
        file_acc[i] = [train_set[i], file_acc[i][0]/file_acc[i][1]*100]
    return loss_/len(trainloader), correct/len(trainloader)*100, file_acc

def eval_(loader, mode='test'):
    file_acc = {}
    correct, loss_ = 0, 0
    model.eval()
    outs, labs = [], []
    for i, data in enumerate(loader, 0):
        if mode=='test':
            inputs, labels, f = data
        else:
            inputs, labels = data
        if cuda_enabled==True:
            inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        else:
            inputs, labels = inputs, labels.squeeze()
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        comp = equals.squeeze().cpu().detach().numpy()
        if mode=='test':
            for j in range(len(f)):
                file = f[j].item()
                if file not in file_acc:
                    file_acc[file] = [0, 0]
                if comp[j] == True:
                    file_acc[file][0]+=1
                    file_acc[file][1]+=1
                else:
                    file_acc[file][1]+=1
    if mode=='test':
        for i in file_acc.keys():
            file_acc[i] = [test_set[i-len(train_set)], file_acc[i][0]/file_acc[i][1]*100]
        return loss_/len(loader), correct/len(loader)*100, file_acc
    else:
        return loss_/len(loader), correct/len(loader)*100

In [ ]:
cuda_enabled = True
#train with hyperparameter tuning on validation set
it = 0
for ss in [15, 30, 45]:
    for feat in [8, 10, 12]:
        for k in [5, 10, 20]:
            for s in [1, k, k//2]:
                for p in [2, 3]:
                    avg = []
                    it += 1
                    for fold in ['1', '2', '3', '4', '5']:
                        
                        total_it = 3*3*3*3*2*5
                        start = time.time()
                        ep = 0
                        all_data = load_data(chunk_s=300, slide_s=ss, fold=fold, norm='inside_chunking', load_dump=True)
                        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label = all_data
                        z = int(((len(data_train[0])-k)/s+1)//p)
                        if cuda_enabled==True:
                            model = Net(feat, k, s, p, z).cuda()
                        else:
                            model = Net(feat, k, s, p, z)
                        criterion = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=0.01)
                        train_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_train)), torch.tensor(np.array(label_train)),torch.tensor(np.array(train_f)))
                        trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64)
                        val_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_val)), torch.tensor(np.array(label_val)))
                        valloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=64)
                        
                        max_train_acc, max_val_acc = 0, 0
                        for epoch in range(100):
                            
                            loss, tr_accuracy, file_accuracy = train()
                            loss, accuracy = eval_(valloader, 'val')
                            
                            if accuracy>max_val_acc: 
                                max_train_acc = tr_accuracy
                                max_val_acc = accuracy 
                                ep = epoch
                                torch.save(model.state_dict(), 'cnnf8k5s1_f{}_w10s_s1s_.pth'.format(fold))
                            elif epoch-ep > 40: 
                                break
                        avg.append(max_val_acc)
                        print('{}/{}, fold {}, train acc: {}, val acc: {}, '.format(it, total_it, fold, round(max_train_acc, 2), round(max_val_acc, 2)), ss, feat, k, s, p, ep, round(time.time() - start, 2))
                    print('avg: ', round(sum(avg)/5, 2))

1/810, fold 1, train acc: 83.42, val acc: 95.94,  15 8 5 1 2 90 60.77
1/810, fold 2, train acc: 77.21, val acc: 93.09,  15 8 5 1 2 18 29.46
1/810, fold 3, train acc: 78.4, val acc: 95.49,  15 8 5 1 2 33 39.14
1/810, fold 4, train acc: 84.11, val acc: 93.6,  15 8 5 1 2 79 44.41
1/810, fold 5, train acc: 82.66, val acc: 84.32,  15 8 5 1 2 49 40.32
avg:  92.49
2/810, fold 1, train acc: 84.35, val acc: 95.31,  15 8 5 1 3 76 43.54
2/810, fold 2, train acc: 74.71, val acc: 60.48,  15 8 5 1 3 54 40.06
2/810, fold 3, train acc: 80.27, val acc: 66.78,  15 8 5 1 3 70 44.77
2/810, fold 4, train acc: 75.42, val acc: 60.4,  15 8 5 1 3 23 27.66
2/810, fold 5, train acc: 58.59, val acc: 100.0,  15 8 5 1 3 0 18.05
avg:  76.59
3/810, fold 1, train acc: 79.27, val acc: 58.91,  15 8 5 5 2 28 27.76
3/810, fold 2, train acc: 78.53, val acc: 86.21,  15 8 5 5 2 28 28.48
3/810, fold 3, train acc: 79.32, val acc: 94.75,  15 8 5 5 2 18 25.28
3/810, fold 4, train acc: 78.2, val acc: 89.95,  15 8 5 5 2 14 23.71
3

23/810, fold 3, train acc: 85.35, val acc: 99.4,  15 10 5 2 2 60 46.26
23/810, fold 4, train acc: 81.25, val acc: 89.65,  15 10 5 2 2 14 26.86
23/810, fold 5, train acc: 80.28, val acc: 79.56,  15 10 5 2 2 14 26.33
avg:  86.4
24/810, fold 1, train acc: 83.05, val acc: 92.27,  15 10 5 2 3 84 45.3
24/810, fold 2, train acc: 77.88, val acc: 94.63,  15 10 5 2 3 16 25.86
24/810, fold 3, train acc: 80.96, val acc: 96.25,  15 10 5 2 3 10 23.96
24/810, fold 4, train acc: 79.54, val acc: 92.77,  15 10 5 2 3 8 23.04
24/810, fold 5, train acc: 86.13, val acc: 82.04,  15 10 5 2 3 98 48.27
avg:  91.59
25/810, fold 1, train acc: 84.96, val acc: 96.25,  15 10 10 1 2 46 39.32
25/810, fold 2, train acc: 82.15, val acc: 91.83,  15 10 10 1 2 35 36.34
25/810, fold 3, train acc: 85.74, val acc: 90.83,  15 10 10 1 2 58 48.37
25/810, fold 4, train acc: 79.95, val acc: 88.49,  15 10 10 1 2 12 25.47
25/810, fold 5, train acc: 85.44, val acc: 92.63,  15 10 10 1 2 69 47.3
avg:  92.01
26/810, fold 1, train acc: 8

45/810, fold 3, train acc: 70.62, val acc: 99.48,  15 12 10 10 2 0 39.88
45/810, fold 4, train acc: 81.84, val acc: 84.29,  15 12 10 10 2 44 70.54
45/810, fold 5, train acc: 81.63, val acc: 78.18,  15 12 10 10 2 9 36.79
avg:  82.85
46/810, fold 1, train acc: 85.01, val acc: 90.39,  15 12 10 10 3 93 81.1
46/810, fold 2, train acc: 76.75, val acc: 61.73,  15 12 10 10 3 19 52.86
46/810, fold 3, train acc: 77.35, val acc: 62.99,  15 12 10 10 3 11 41.72
46/810, fold 4, train acc: 81.32, val acc: 86.23,  15 12 10 10 3 61 89.92
46/810, fold 5, train acc: 82.32, val acc: 81.39,  15 12 10 10 3 29 51.95
avg:  76.55
47/810, fold 1, train acc: 82.97, val acc: 82.19,  15 12 10 5 2 44 70.52
47/810, fold 2, train acc: 81.98, val acc: 90.59,  15 12 10 5 2 85 71.53
47/810, fold 3, train acc: 85.43, val acc: 95.34,  15 12 10 5 2 72 82.72
47/810, fold 4, train acc: 85.85, val acc: 94.37,  15 12 10 5 2 29 52.73
47/810, fold 5, train acc: 82.39, val acc: 83.25,  15 12 10 5 2 44 69.48
avg:  89.15
48/810, fo

67/810, fold 4, train acc: 80.43, val acc: 88.1,  30 8 20 1 2 17 25.43
67/810, fold 5, train acc: 78.36, val acc: 63.51,  30 8 20 1 2 14 23.63
avg:  72.77
68/810, fold 1, train acc: 82.79, val acc: 77.18,  30 8 20 1 3 14 22.28
68/810, fold 2, train acc: 78.74, val acc: 91.69,  30 8 20 1 3 1 17.59
68/810, fold 3, train acc: 86.77, val acc: 79.28,  30 8 20 1 3 89 41.74
68/810, fold 4, train acc: 85.13, val acc: 75.36,  30 8 20 1 3 73 45.48
68/810, fold 5, train acc: 85.0, val acc: 88.97,  30 8 20 1 3 19 27.23
avg:  82.5
69/810, fold 1, train acc: 79.99, val acc: 59.29,  30 8 20 20 2 95 40.11
69/810, fold 2, train acc: 78.42, val acc: 85.75,  30 8 20 20 2 24 27.86
69/810, fold 3, train acc: 82.38, val acc: 86.17,  30 8 20 20 2 20 24.67
69/810, fold 4, train acc: 83.24, val acc: 86.17,  30 8 20 20 2 71 41.3
69/810, fold 5, train acc: 75.87, val acc: 58.42,  30 8 20 20 2 4 20.26
avg:  75.16
70/810, fold 1, train acc: 81.29, val acc: 71.54,  30 8 20 20 3 51 38.71
70/810, fold 2, train acc: 8

89/810, fold 4, train acc: 83.97, val acc: 73.48,  30 10 20 10 2 94 35.92
89/810, fold 5, train acc: 75.97, val acc: 60.13,  30 10 20 10 2 2 17.34
avg:  73.2
90/810, fold 1, train acc: 83.23, val acc: 80.79,  30 10 20 10 3 65 28.82
90/810, fold 2, train acc: 82.06, val acc: 80.38,  30 10 20 10 3 56 31.11
90/810, fold 3, train acc: 82.48, val acc: 83.28,  30 10 20 10 3 23 23.72
90/810, fold 4, train acc: 83.7, val acc: 76.26,  30 10 20 10 3 86 31.28
90/810, fold 5, train acc: 84.0, val acc: 72.55,  30 10 20 10 3 48 32.36
avg:  78.65
91/810, fold 1, train acc: 83.61, val acc: 83.09,  30 12 5 1 2 73 28.8
91/810, fold 2, train acc: 76.71, val acc: 85.29,  30 12 5 1 2 6 17.82
91/810, fold 3, train acc: 78.17, val acc: 92.94,  30 12 5 1 2 39 28.73
91/810, fold 4, train acc: 80.82, val acc: 84.47,  30 12 5 1 2 21 22.66
91/810, fold 5, train acc: 68.88, val acc: 84.26,  30 12 5 1 2 0 13.57
avg:  86.01
92/810, fold 1, train acc: 82.19, val acc: 88.5,  30 12 5 1 3 89 26.73
92/810, fold 2, train 

In [ ]:
#20, 32!, 47, [till 91]

In [5]:
avg = []
ss = 30; k = 10; feat = 12; s = 10; p =2;
#model results on test set
for fold in ['1', '2', '3', '4', '5']:
    start = time.time()
    ep = 0
    data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, full_label = load_data(chunk_s=300, slide_s=ss, fold=fold, norm='inside_chunking')
    z = int(((len(data_train[0])-k)/s+1)//p)
    if cuda_enabled==True:
        model = Net(feat, k, s, p, z).cuda()
    else:
        model = Net(feat, k, s, p, z)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_train)), torch.tensor(np.array(label_train)),torch.tensor(np.array(train_f)))
    trainloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64)
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_test)), torch.tensor(np.array(label_test)), torch.tensor(np.array(test_f)))
    testloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=64)
    max_train_acc, max_test_acc = 0, 0
    for epoch in range(100):
        loss, tr_accuracy, file_accuracy = train()
        if accuracy>max_train_acc: max_train_acc = accuracy
        if accuracy>max_test_acc: 
            max_train_acc = tr_accuracy
            max_test_acc = accuracy 
            ep = epoch
        elif epoch-ep > 40: 
            break
    avg.append(max_test_acc)
    print('fold {}, train acc: {}, test acc: {}, '.format(fold, round(max_train_acc, 2), round(max_test_acc, 2)), ss, feat, k, s, p, ep, time.time() - start)
print('avg: ', round(sum(avg)/5, 2))

NameError: name 'cuda_enabled' is not defined

In [ ]:
#get majority voting(file wise) values
fold = '1'
ss = 30
data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
test_dataset = torch.utils.data.TensorDataset(torch.tensor(np.array(data_test)), torch.tensor(np.array(label_test)), torch.tensor(np.array(test_f)))
testloader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=128)
from sklearn.metrics import confusion_matrix
feat, k, s, p = 12, 10, 10, 2
z = int(((len(data_train[0])-k)/s+1)//p)
model = Net(feat, k, s, p, z).cuda()
model.load_state_dict(torch.load('cnnf8k5s1_f{}_w10s_s1s_.pth'.format(fold)))
import matplotlib.pyplot as plt
import sklearn
#plot kernels 
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         if name == 'cnn.conv.weight':
#             data = param.data
# #             data = data.permute(2, 1, 0)
#             for i in range(12):
#                 plt.scatter(data[i][0].detach().cpu().numpy())
#                 plt.show()
#         #         plt.plot(param.data.detach().cpu().numpy())


criterion = nn.CrossEntropyLoss()
# model.load_state_dict('cnnf8k5s1_f1_w10s_s0.5s_.pth')
model.eval()
def eval_():
    file_acc_ = {}
    file_acc = {}
    model.eval()
    correct = 0
    loss_ = 0
    outs = []
    labs = []
    mv_count = [0]*11
    for i, data in enumerate(testloader, 0):
        
        inputs, labels, f = data
        inputs, labels = inputs.cuda(), labels.cuda().squeeze()
        
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)        
        loss_ +=  loss.item()
        softmax_output = torch.exp(outputs)
        top_p, top_class = softmax_output.topk(1, dim=1)
        outs.extend(top_class.view(top_class.shape[0]).cpu().detach().numpy().tolist())
        labs.extend(labels.cpu().detach().numpy().tolist())
        equals = top_class == labels.view(*top_class.shape)
        correct += torch.mean(equals.type(torch.FloatTensor)).item()
        comp = equals.squeeze().cpu().detach().numpy()
        for j in range(len(f)):
            file = f[j].item()
            if file not in file_acc:
                file_acc[file] = []
            file_acc[file].append(top_class[j].cpu().detach().numpy())
            if file not in file_acc_:
                file_acc_[file] = [0, 0]
            if comp[j] == True:
                file_acc_[file][0]+=1
                file_acc_[file][1]+=1
            else:
                file_acc_[file][1]+=1
    for i in file_acc_.keys():
        file_acc_[i] = [test_set[i-len(train_set)], file_acc_[i][0]/file_acc_[i][1]*100]
    cd = [0]*11
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    for j in range(1, 11):
        cl = []
        for i in file_acc.keys():
            l = len(file_acc[i])
            if l>9:
                seg = file_acc[i][:j*l//10]
            else:
                if j<=l:
                    seg = file_acc[i][:j*l//l]
                else:
                    seg = file_acc[i]
            seg_acc = sum(seg)/len(seg)
            if seg_acc > 0.5: #more ones than zeros
                cl.append(1)
            else:
                cl.append(0) #more zeros than ones
        print(round(accuracy_score(f_l, cl), 4))
    print('~~~~')
    for j in range(1, 11):
        cl = []
        for i in file_acc.keys():
            l = len(file_acc[i])
            if l>9:
                seg = file_acc[i][:j*l//10]
            else:
                if j<=l:
                    seg = file_acc[i][:j*l//l]
                else:
                    seg = file_acc[i]
            seg_acc = sum(seg)/len(seg)
            if seg_acc > 0.5: #more ones than zeros
                cl.append(1)
            else:
                cl.append(0) #more zeros than ones
        
        print(round(f1_score(f_l, cl), 4))
        print('majority:',sklearn.metrics.confusion_matrix(f_l, cl))
    print('chunkwise:',round(accuracy_score(labs, outs), 4))
    print('chunkwise:',sklearn.metrics.confusion_matrix(labs, outs))
    return loss_/len(testloader), correct/len(testloader)*100,  outs, labs, file_acc_
a = eval_()
a[-1]

In [ ]:
def get_random():
    r = np.random.randint(0, len(data_train))
    plt.plot(data_train[r])
    print(label_train[r])
get_random()

In [ ]:
fold = '5'
ss = 30
import sklearn.svm
from sklearn import metrics

In [ ]:
def get_svm_results_on_val(svm):
        fold = '1'
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_val))
        print(fold, metrics.accuracy_score(np.array(label_val).ravel(), y_pred))
        
for gamma in ['scale', 'auto']:
    for degree in [2, 3, 4]:
        for coef in [0, 0.5, 1, 5]:
            for C in [0.1, 0.5, 1, 2]:
                for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
                    print(gamma, degree, coef, C, kernel)
                    svm = sklearn.svm.SVC(kernel=kernel, gamma=gamma, coef0=coef, C=C, degree=degree)
                    get_svm_results_on_val(svm)

In [ ]:
def get_svm_results_on_val(svm, fold='1'):
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_val))
        print(fold, metrics.accuracy_score(np.array(label_val).ravel(), y_pred))
for gamma in ['auto']:
    for degree in [3, 4]:
        for coef in [0, 0.5, 1, 5]:
            for C in [0.1, 0.5, 1, 2]:
                for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
                    print(gamma, degree, coef, C, kernel)
                    svm = sklearn.svm.SVC(kernel=kernel, gamma=gamma, coef0=coef, C=C, degree=degree)
                    get_svm_results_on_val(svm)

In [ ]:
def get_svm_results_on_test(svm, fold='1'):
        data_train, label_train, data_test, label_test, data_val, label_val, train_f, test_f, train_set, test_set, chunks_count, f_l = load_data(chunk_s=300, slide_s=ss, fold=fold, norm=True)
        svm.fit(np.squeeze(data_train), np.array(label_train).ravel())
        y_pred = svm.predict(np.squeeze(data_test))
        print(fold, metrics.accuracy_score(np.array(label_test).ravel(), y_pred))
for fold in ['1', '2', '3', '4', '5']:
        get_svm_results_on_test(svm, fold)